In [3]:
import sqlite3
from sqlite3 import Error
import json

In [4]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_client_periods_table(conn):
    create_table_sql="""CREATE TABLE IF NOT EXISTS clients_periods (
	id INTEGER PRIMARY KEY,
	period_end TEXT NOT NULL,
	client_id TEXT NOT NULL,
	attributed_customers INTEGER,
	unattributed_customers INTEGER,
    contribution_rate REAL
);"""
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def create_records_table(conn):
    create_table_sql="""
    CREATE TABLE IF NOT EXISTS records(
        id INTEGER PRIMARY KEY,
        source TEXT NOT NULL,
        source_type TEXT NOT NULL,
        attributed BOOL,
        customers INTEGER
    );
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def list_tables(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        return [table[0] for table in tables]
    except sqlite3.Error as e:
        print("Error:", e)
        return []

conn = create_connection("/home/cristi/Documents/practica/database/ClientDB/Clients.db")
create_client_periods_table(conn)
create_records_table(conn)
tables = list_tables(conn)
for table in tables:
    print(table)

clients_periods
records


In [5]:
def insert_to_clients_periods(conn,clientid,period,att,unatt,cr):
    insertion_str="""INSERT INTO clients_periods VALUES (? ? ? ? ?)"""
    

In [6]:
def load_from_json_to_db(conn):
    client_records=[]
    with open("/home/cristi/Documents/practica/clients_per_day.json",'r') as f:
        client_data=json.load(f)
    for client in client_data:
        print(client)
        for period in client_data[client]:
            print(period)
            for attributions in client_data[client][period]:
                attributed=client_data[client][period]["ATTRIBUTED"]
                unattributed=client_data[client][period]["UNATTRIBUTED"]
                cr=client_data[client][period]["CR"]
            print(attributed,unattributed,cr)
        client_record=(client,period,attributed,unattributed,cr)
        client_records.append(client_record)
    return client_records

In [7]:
load_from_json_to_db(conn)

        


1446518931
2023-01-02T00:00:00
0 3027 0.0
2023-01-09T00:00:00
0 1093 0.0
2023-01-16T00:00:00
0 2128 0.0
2023-01-23T00:00:00
0 2324 0.0
2023-01-30T00:00:00
0 1254 0.0
2023-02-06T00:00:00
0 1308 0.0
2023-02-13T00:00:00
0 2593 0.0
2023-02-20T00:00:00
0 3119 0.0
2023-02-27T00:00:00
0 1297 0.0
2023-03-06T00:00:00
0 1516 0.0
2023-03-13T00:00:00
0 3068 0.0
2023-03-20T00:00:00
0 3076 0.0
2023-03-27T00:00:00
0 1179 0.0
2023-04-03T00:00:00
0 1170 0.0
2023-04-10T00:00:00
0 1207 0.0
2023-04-17T00:00:00
0 2987 0.0
2023-04-24T00:00:00
0 3632 0.0
2023-05-01T00:00:00
0 1551 0.0
2023-05-08T00:00:00
0 1488 0.0
2023-05-15T00:00:00
0 1582 0.0
2023-05-22T00:00:00
0 4135 0.0
2023-05-29T00:00:00
0 4814 0.0
2023-06-05T00:00:00
0 1396 0.0
2023-06-12T00:00:00
0 2051 0.0
2023-06-19T00:00:00
0 1702 0.0
2023-06-26T00:00:00
0 3472 0.0
2023-07-03T00:00:00
0 5009 0.0
2023-07-10T00:00:00
0 1524 0.0
2023-07-17T00:00:00
0 1531 0.0
2023-07-24T00:00:00
0 1763 0.0
2023-07-31T00:00:00
0 2994 0.0
2023-08-07T00:00:00
0 3101 0


0 7859 0.0
2023-05-01T00:00:00
0 7387 0.0
2023-05-08T00:00:00
0 7092 0.0
2023-05-15T00:00:00
0 8192 0.0
2023-05-22T00:00:00
0 8032 0.0
2023-05-29T00:00:00
0 8101 0.0
2023-06-05T00:00:00
0 8641 0.0
2023-06-12T00:00:00
0 8614 0.0
2023-06-19T00:00:00
0 9780 0.0
2023-06-26T00:00:00
0 9339 0.0
2023-07-03T00:00:00
0 9316 0.0
2023-07-10T00:00:00
0 10037 0.0
2023-07-17T00:00:00
0 9692 0.0
2023-07-24T00:00:00
0 10482 0.0
2023-07-31T00:00:00
0 10663 0.0
2023-08-07T00:00:00
0 10347 0.0
2023-08-14T00:00:00
0 10667 0.0
2023-08-21T00:00:00
0 12096 0.0
2023-08-28T00:00:00
0 11544 0.0
2023-09-04T00:00:00
0 11212 0.0
2023-09-11T00:00:00
0 10282 0.0
2023-09-18T00:00:00
0 1571 0.0
21760749
2023-01-02T00:00:00
0 11760 0.0
2023-01-09T00:00:00
0 12393 0.0
2023-01-16T00:00:00
0 11681 0.0
2023-01-23T00:00:00
0 12851 0.0
2023-01-30T00:00:00
0 11699 0.0
2023-02-06T00:00:00
0 11716 0.0
2023-02-13T00:00:00
0 13794 0.0
2023-02-20T00:00:00
0 13065 0.0
2023-02-27T00:00:00
0 10507 0.0
2023-03-06T00:00:00
0 9988 0.0


[('1446518931', '2023-09-18T00:00:00', 0, 194, 0.0),
 ('1736295694', '2023-09-18T00:00:00', 0, 420, 0.0),
 ('160484', '2023-03-06T00:00:00', 0, 140, 0.0),
 ('944738563', '2023-09-18T00:00:00', 0, 4, 0.0),
 ('217783280', '2023-01-16T00:00:00', 0, 5225, 0.0),
 ('1352108802', '2023-09-11T00:00:00', 0, 5842, 0.0),
 ('62816704', '2023-09-11T00:00:00', 0, 175, 0.0),
 ('1496345254', '2023-09-18T00:00:00', 0, 1616, 0.0),
 ('21243602', '2023-09-18T00:00:00', 0, 204, 0.0),
 ('1288402200', '2023-09-18T00:00:00', 0, 6650, 0.0),
 ('668700215', '2023-09-18T00:00:00', 0, 559, 0.0),
 ('86545', '2023-09-11T00:00:00', 0, 6028, 0.0),
 ('2113150881', '2023-09-18T00:00:00', 0, 118, 0.0),
 ('1933045607', '2023-09-18T00:00:00', 0, 1424, 0.0),
 ('447268576', '2023-09-18T00:00:00', 0, 72, 0.0),
 ('1544104793', '2023-09-11T00:00:00', 0, 57, 0.0),
 ('377235927', '2023-09-11T00:00:00', 0, 2207, 0.0),
 ('1816143665', '2023-09-18T00:00:00', 0, 938, 0.0),
 ('1988299953', '2023-09-18T00:00:00', 0, 635, 0.0),
 ('20363